# Model South Korea Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.data_plots_kr import load_df_feat_kr
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

from my_helpers.model_kr import prepare_data_features_kr
from my_helpers.model_kr import prepare_dataset_kr

from my_helpers.utils import clean_file

import settings

### Definitions

In [2]:
PATH_TO_SAVE_DATA = settings.PATH_TO_SAVE_DATA

from my_helpers.model_kr import PATH_MDL_MULTI_STEP_KR
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_KR 
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_FILE_KR
from my_helpers.model_kr import PATH_SERVERLESS_KR
from my_helpers.model_kr import URL_PREDICT_KR
from my_helpers.model_kr import PAST_HISTORY # days used to predict next values in future
from my_helpers.model_kr import FUTURE_TARGET  # predict 3 days later
from my_helpers.model_kr import STEP 

from my_helpers.data_plots_kr import PATH_DF_FEAT_KR

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model_kr import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")


TRAIN_SPLIT = 326


### helper functions

### load data

In [3]:
# load
df_feat_kr = load_df_feat_kr()
# clean
df_feat_kr = prepare_data_features_kr(df_feat_kr)
# split train/test
df_feat_kr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_kr.shape[0])]
df_feat_kr

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2020-04-03,10062.0,424365.0,174.0,2020-04-03,5.0,0.0,1.0,5.0,1.0,1.0,...,5.000000,10.0,inf,106.0,4.240000,20.0,20.000000,5.0,inf,True
2020-04-04,10156.0,434888.0,177.0,2020-04-04,6.0,0.0,1.0,1.0,0.0,0.0,...,5.000000,10.0,inf,67.0,1.030769,21.0,21.000000,5.0,inf,True
2020-04-05,10237.0,441662.0,183.0,2020-04-05,0.0,3.0,1.0,4.0,0.0,1.0,...,6.000000,9.0,9.000000,60.0,0.789474,21.0,10.500000,8.0,inf,True
2020-04-06,10284.0,447509.0,186.0,2020-04-06,1.0,0.0,2.0,2.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.769231,22.0,7.333333,8.0,inf,True
2020-04-07,10331.0,456654.0,192.0,2020-04-07,2.0,0.0,1.0,1.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.759494,22.0,5.500000,8.0,inf,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-04,91239.0,6691811.0,1619.0,2021-03-04,4.0,3.0,10.0,7.0,1.0,6.0,...,2.320000,72.0,1.846154,162.0,1.760870,79.0,0.908046,29.0,1.115385,False
2021-03-05,91637.0,6725304.0,1627.0,2021-03-05,5.0,2.0,3.0,6.0,0.0,3.0,...,2.188679,62.0,1.265306,146.0,1.364486,73.0,0.811111,28.0,1.037037,False
2021-03-06,92055.0,6756772.0,1632.0,2021-03-06,6.0,5.0,5.0,13.0,3.0,4.0,...,1.964912,55.0,0.948276,140.0,1.176471,70.0,0.813953,31.0,1.347826,False


In [4]:
df_feat_kr[df_feat_kr["train"] == False].head(5)

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2021-02-24,88120.0,6436000.0,1576.0,2021-02-24,3.0,2.0,8.0,18.0,1.0,8.0,...,2.135135,70.0,2.058824,149.0,0.805405,88.0,0.642336,24.0,1.142857,False
2021-02-25,88516.0,6482542.0,1581.0,2021-02-25,4.0,5.0,2.0,9.0,11.0,5.0,...,1.790698,81.0,3.115385,157.0,1.097902,85.0,0.691057,25.0,1.000000,False
2021-02-26,88922.0,6521124.0,1585.0,2021-02-26,5.0,0.0,6.0,24.0,5.0,16.0,...,2.333333,82.0,3.280000,177.0,1.701923,87.0,0.776786,21.0,0.750000,False
2021-02-27,89317.0,6558225.0,1595.0,2021-02-27,6.0,1.0,1.0,7.0,5.0,20.0,...,2.585366,87.0,3.625000,179.0,2.057471,84.0,0.807692,22.0,0.846154,False
2021-02-28,89672.0,6576114.0,1603.0,2021-02-28,0.0,1.0,2.0,4.0,0.0,7.0,...,2.377778,85.0,3.400000,178.0,2.342105,80.0,0.869565,23.0,0.920000,False


### Prepare dataset

In [5]:
dataset, data_std, data_mean = prepare_dataset_kr(df_feat_kr)
print(dataset)
print("data_mean : ", data_mean)
print("data_std : ", data_std)

[[-0.47702324 -0.07464522 -1.1302585  ... -0.49368896  0.99618756
  -0.4174528 ]
 [-0.47382569 -0.16217883 -1.29915966 ... -0.49266611  1.49428134
  -0.4174528 ]
 [-0.72643203 -0.50531056 -1.80400453 ... -0.73233384 -1.49428134
  -0.4174528 ]
 ...
 [-0.56335705 -0.85894633  0.87936329 ...  0.84631511  1.49428134
  -5.3578543 ]
 [-0.60172764 -0.93597591 -0.199792   ...  0.11049874 -1.49428134
  -4.72127202]
 [-0.94706288 -0.61735358  0.49277246 ... -0.03481061 -0.99618756
  -0.42098637]]
data_mean :  [1.02061350e+01 1.95439673e+01 6.28565440e+01 1.83586912e+01
 2.37263804e+02 1.82295245e+04 3.00000000e+00 4.43787904e+01]
data_std :  [1.04246527e+01 9.52015313e+00 1.43476417e+01 6.30313145e+00
 3.25978900e+02 1.56424897e+04 2.00765407e+00 5.94228918e+00]


In [6]:
dataset.shape[1]

8

In [7]:
PAST_HISTORY

14

In [8]:
dataset.shape[1]

8

### Load model

In [9]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP_KR)


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x639e3c390> and <tensorflow.python.keras.layers.core.Dropout object at 0x639eb23d0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x639eb2c50> and <tensorflow.python.keras.layers.core.Dense object at 0x639ec31d0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x639eb2c50> and <tensorflow.python.keras.layers.core.Dropout object at 0x639eb23d0>).
CPU times: user 2.27 s, sys: 149 ms, total: 2.42 s
Wall time: 2.62 s


In [10]:
multi_step_model.inputs[0].dtype

tf.float32

In [11]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_lstm"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/keras_lstm/assets


### Save model TFlite

In [12]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE_KR)

File /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model_20210308_16_33_27.tflite moved!


In [13]:
open(PATH_MDL_MULTI_TFLITE_FILE_KR, "wb").write(tflite_model)

5500

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [14]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[263 - 277]
[270 - 284]
[277 - 291]
[284 - 298]
[291 - 305]
[298 - 312]
[305 - 319]
[312 - 326]
[319 - 333]
9


array([[702.86414, 729.0875 , 736.69977, 688.97797, 653.7916 , 631.0095 ,
        626.08215, 470.4515 , 481.029  , 487.1847 , 470.63177, 457.53943,
        451.49536, 451.31912, 401.8062 , 414.47073, 419.2056 , 399.11536,
        384.37756, 376.51666, 375.6153 , 398.42123, 416.94165, 425.10516,
        409.0271 , 397.15097, 390.36792, 388.54022, 375.80865, 397.06296,
        410.5708 , 401.27368, 393.4677 , 388.4569 , 385.42538, 355.71664,
        357.49164, 356.8145 , 343.8663 , 335.03714, 332.2276 , 333.76288,
        424.83344, 442.50467, 456.15222, 451.4508 , 446.40405, 443.33276,
        440.14886, 357.32477, 380.7345 , 394.64774, 386.7549 , 379.08932,
        373.8289 , 369.7801 , 382.63135, 408.12372, 427.1452 , 423.6209 ,
        419.92017, 416.48627, 412.22656]], dtype=float32)

#### Future days

In [15]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [16]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [17]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE_KR)

In [18]:
PATH_MDL_MULTI_TFLITE_FILE_KR

'/Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model.tflite'

### Predict reloaded model

#### Past days

In [19]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [20]:
x_multi.shape

(1, 14, 8)

In [21]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [22]:
dataset.shape

(340, 8)

In [23]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[263 - 277]
[270 - 284]
[277 - 291]
[284 - 298]
[291 - 305]
[298 - 312]
[305 - 319]
[312 - 326]
[319 - 333]


In [24]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda_kr.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [25]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [26]:
y_multi_pred

array([[1.4283142 , 1.5087593 , 1.5321113 , 1.385716  , 1.2777754 ,
        1.207887  , 1.1927714 , 0.715346  , 0.7477943 , 0.7666781 ,
        0.715899  , 0.67573583, 0.6571945 , 0.6566539 , 0.50476396,
        0.5436147 , 0.55813974, 0.49650925, 0.4512984 , 0.42718363,
        0.42441848, 0.4943799 , 0.5511947 , 0.5762378 , 0.5269154 ,
        0.49048316, 0.46967486, 0.46406808, 0.42501172, 0.49021313,
        0.53165096, 0.5031304 , 0.47918403, 0.46381247, 0.45451275,
        0.36337575, 0.36882085, 0.3667437 , 0.32702264, 0.29993758,
        0.29131883, 0.2960286 , 0.57540417, 0.62961394, 0.67148024,
        0.6570579 , 0.64157605, 0.6321542 , 0.6223871 , 0.36830896,
        0.44012263, 0.48280403, 0.45859134, 0.43507576, 0.4189384 ,
        0.40651798, 0.44594154, 0.5241441 , 0.5824959 , 0.5716846 ,
        0.56033176, 0.54979765, 0.53673035]], dtype=float32)

In [27]:
y_multi_pred_out

array([[1.42831445, 1.50875938, 1.53211153, 1.38571608, 1.27777541,
        1.20788717, 1.19277155, 0.71534598, 0.74779433, 0.76667815,
        0.71589893, 0.67573583, 0.65719444, 0.65665388, 0.50476408,
        0.54361475, 0.55813986, 0.49650931, 0.45129839, 0.42718372,
        0.42441854, 0.49437991, 0.55119479, 0.57623786, 0.52691537,
        0.49048316, 0.46967489, 0.46406808, 0.42501178, 0.49021319,
        0.53165096, 0.50313044, 0.47918409, 0.46381253, 0.4545128 ,
        0.36337578, 0.36882088, 0.36674371, 0.32702273, 0.29993764,
        0.29131889, 0.29602864, 0.57540423, 0.62961388, 0.67148024,
        0.65705794, 0.64157605, 0.63215423, 0.62238711, 0.36830893,
        0.44012263, 0.48280394, 0.45859128, 0.43507576, 0.41893834,
        0.40651795, 0.44594151, 0.52414411, 0.58249593, 0.5716846 ,
        0.56033182, 0.54979759, 0.53673035]])

In [28]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [29]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [30]:
y_future_pred_out

array([[0.40668079, 0.47320628, 0.51792705, 0.4898783 , 0.46490002,
        0.44769371, 0.43746027]])

In [31]:
y_future_pred

array([[0.4066808 , 0.47320625, 0.51792705, 0.4898783 , 0.46490002,
        0.44769368, 0.43746024]], dtype=float32)

In [32]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda-kr
    
- Execute : sls deploy -v

In [33]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE_KR}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr\nserverless deploy -v'

In [34]:
open('deploy_serverless_kr.sh', "w").write(str_exe)
os.chmod('deploy_serverless_kr.sh', stat.S_IRWXU)

In [35]:
!cat ./deploy_serverless_kr.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr
serverless deploy -v

In [36]:
!./deploy_serverless_kr.sh

Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/ef4e42eb03bbad46f74fee99a3c01f994e8119e7a557f947779093ab37b248d3_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: Injecting required Python packages to package...
Serverless: Uploading CloudFormation file to S3...
Serverless: Uploading artifacts...
Serverless: Uploading service tensorflow-lite-on-aws-lambda-kr.zip file to S3 (19.22 MB)...
Serverless: Validating template...
Serverless: Updating Stack...
Serverless: Checking Stack update progress

### API AWS real Test

#### Past days

In [37]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[263 - 277]
[270 - 284]
[277 - 291]
[284 - 298]
[291 - 305]
[298 - 312]
[305 - 319]
[312 - 326]
[319 - 333]
status code :  200
[[[1.428314447402954, 1.5087592601776123, 1.5321115255355835, 1.3857159614562988, 1.2777754068374634, 1.2078871726989746, 1.1927714347839355]], [[0.7153458595275879, 0.7477942109107971, 0.7666780352592468, 0.715898871421814, 0.6757358312606812, 0.6571943759918213, 0.656653881072998]], [[0.5047640204429626, 0.5436147451400757, 0.5581398010253906, 0.49650925397872925, 0.45129841566085815, 0.42718368768692017, 0.42441853880882263]], [[0.4943799078464508, 0.5511947274208069, 0.5762378573417664, 0.5269153714179993, 0.4904831647872925, 0.46967485547065735, 0.46406808495521545]], [[0.4250117838382721, 0.49021318554878235, 0.531650960445404, 0.5031304359436035, 0.479184091091156, 0.4638124704360962, 0.4545128047466278]], [[0.3633757531642914, 0.36882084608078003, 0.36674371361732483, 0.3270227015018463, 0.2999376058578491, 0.2913188338279724, 0.29602858424186707]], [[0

In [38]:
len(json_list_list_x)

20683

In [39]:
resp.json()

[[[1.428314447402954,
   1.5087592601776123,
   1.5321115255355835,
   1.3857159614562988,
   1.2777754068374634,
   1.2078871726989746,
   1.1927714347839355]],
 [[0.7153458595275879,
   0.7477942109107971,
   0.7666780352592468,
   0.715898871421814,
   0.6757358312606812,
   0.6571943759918213,
   0.656653881072998]],
 [[0.5047640204429626,
   0.5436147451400757,
   0.5581398010253906,
   0.49650925397872925,
   0.45129841566085815,
   0.42718368768692017,
   0.42441853880882263]],
 [[0.4943799078464508,
   0.5511947274208069,
   0.5762378573417664,
   0.5269153714179993,
   0.4904831647872925,
   0.46967485547065735,
   0.46406808495521545]],
 [[0.4250117838382721,
   0.49021318554878235,
   0.531650960445404,
   0.5031304359436035,
   0.479184091091156,
   0.4638124704360962,
   0.4545128047466278]],
 [[0.3633757531642914,
   0.36882084608078003,
   0.36674371361732483,
   0.3270227015018463,
   0.2999376058578491,
   0.2913188338279724,
   0.29602858424186707]],
 [[0.575404167175

In [40]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [41]:
y_multi_pred_out

array([[1.42831445, 1.50875926, 1.53211153, 1.38571596, 1.27777541,
        1.20788717, 1.19277143, 0.71534586, 0.74779421, 0.76667804,
        0.71589887, 0.67573583, 0.65719438, 0.65665388, 0.50476402,
        0.54361475, 0.5581398 , 0.49650925, 0.45129842, 0.42718369,
        0.42441854, 0.49437991, 0.55119473, 0.57623786, 0.52691537,
        0.49048316, 0.46967486, 0.46406808, 0.42501178, 0.49021319,
        0.53165096, 0.50313044, 0.47918409, 0.46381247, 0.4545128 ,
        0.36337575, 0.36882085, 0.36674371, 0.3270227 , 0.29993761,
        0.29131883, 0.29602858, 0.57540417, 0.62961388, 0.67148024,
        0.65705794, 0.64157605, 0.63215423, 0.62238711, 0.36830893,
        0.44012263, 0.482804  , 0.45859131, 0.43507579, 0.4189384 ,
        0.40651798, 0.44594151, 0.52414411, 0.58249593, 0.5716846 ,
        0.56033182, 0.54979765, 0.53673041]])

In [42]:
y_multi_pred

array([[1.4283142 , 1.5087593 , 1.5321113 , 1.385716  , 1.2777754 ,
        1.207887  , 1.1927714 , 0.715346  , 0.7477943 , 0.7666781 ,
        0.715899  , 0.67573583, 0.6571945 , 0.6566539 , 0.50476396,
        0.5436147 , 0.55813974, 0.49650925, 0.4512984 , 0.42718363,
        0.42441848, 0.4943799 , 0.5511947 , 0.5762378 , 0.5269154 ,
        0.49048316, 0.46967486, 0.46406808, 0.42501172, 0.49021313,
        0.53165096, 0.5031304 , 0.47918403, 0.46381247, 0.45451275,
        0.36337575, 0.36882085, 0.3667437 , 0.32702264, 0.29993758,
        0.29131883, 0.2960286 , 0.57540417, 0.62961394, 0.67148024,
        0.6570579 , 0.64157605, 0.6321542 , 0.6223871 , 0.36830896,
        0.44012263, 0.48280403, 0.45859134, 0.43507576, 0.4189384 ,
        0.40651798, 0.44594154, 0.5241441 , 0.5824959 , 0.5716846 ,
        0.56033176, 0.54979765, 0.53673035]], dtype=float32)

In [43]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [44]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[0.40668079257011414, 0.4732062816619873, 0.5179270505905151, 0.4898782968521118, 0.46490001678466797, 0.44769370555877686, 0.4374602735042572]]]


In [45]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[0.40668079, 0.47320628, 0.51792705, 0.4898783 , 0.46490002,
        0.44769371, 0.43746027]])

In [46]:
y_future_pred

array([[0.4066808 , 0.47320625, 0.51792705, 0.4898783 , 0.46490002,
        0.44769368, 0.43746024]], dtype=float32)

In [47]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
